# Digital Elevation Model (DEM) Tile Preprocessing

This Jupyter Notebook is dedicated to the initial, essential preprocessing steps required for the 1 arc-second resolution Digital Elevation Model (DEM) tiles covering the Ohio Region study area - 104 in total. Given the computational intensity associated with processing numerous high-resolution raster tiles, a strategy leveraging Virtual Raster Tiles (VRTs) will be employed instead of physically merging the tiles into a single large raster dataset. This approach allows for efficient handling of the data volume by treating the collection of tiles as a single virtual dataset without creating a massive intermediate file.

The preprocessing workflow in this notebook focuses on preparing the DEM data for subsequent watershed delineation, a process that is inherently agnostic to projection at this stage. Therefore, reprojection to a specific projected coordinate system like UTM will not be performed on the raw or preprocessed tiles. Instead, projection will be applied later to the delineated watershed boundaries and derived vector features.

The specific steps undertaken in this notebook are:

1.  **Verification of Coordinate Reference System (CRS):** Ensuring that all individual 1 arc-second DEM tiles are consistently referenced to the NAD83 datum (EPSG:4269), which is the native CRS for this dataset.
2.  **Filling Pits and Depressions:** Applying algorithms to identify and fill sinks and depressions within each individual DEM tile. Sinks (or pits) are local topographic lows surrounded by higher elevation values, where water flow would terminate in a standard flow direction calculation. Filling these depressions creates a hydrologically connected surface by raising the elevation of sink cells to the level of their lowest spill point. This step is critical for accurately determining continuous flow paths across the terrain, which is a necessary prerequisite for deriving flow direction and accumulation grids used in automated watershed delineation.
3.  **Creation of Virtual Raster Tiles (VRTs):** Constructing VRT files that virtually mosaic the preprocessed (sink-filled) individual DEM tiles. These VRTs will serve as the input for subsequent terrain analysis steps, allowing seamless access to the data across tile boundaries without the need for physical mosaicking.

The output of this notebook will be a collection of sink-filled DEM tiles, organized into VRTs, ready for flow direction and accumulation calculations leading to watershed delineation.

In [ ]:
# Import libraries
import rasterio
import richdem as rd
import glob
import os
import numpy as np
from tqdm.notebook import tqdm
from subprocess import call

# Set paths
dem_dir = './dem_tiles'
filled_dir = './dem_filled_tiles'
os.makedirs(filled_dir, exist_ok=True)
original_vrt = 'ohio_dem.vrt'
filled_vrt = 'ohio_dem_filled.vrt'

## Step 1: Verify NAD83 CRS

Check the CRS of each DEM tile to ensure it’s NAD83 (expected: EPSG:4269 for geographic or EPSG:26916/26917 for UTM). Report any tiles with a different CRS for manual inspection.

In [ ]:
# Get list of DEM files
dem_files = glob.glob(f"{dem_dir}/*.tif")
print(f"Found {len(dem_files)} DEM files.")

# Expected NAD83 CRS codes
nad83_crs = ['EPSG:4269', 'EPSG:26916', 'EPSG:26917']

# Check CRS for each tile
mismatched_tiles = []
for dem_file in tqdm(dem_files, desc="Checking CRS"):
    with rasterio.open(dem_file) as src:
        crs = str(src.crs).upper()
        if crs not in nad83_crs:
            mismatched_tiles.append((dem_file, crs))

# Report results
if mismatched_tiles:
    print("\nTiles with non-NAD83 CRS:")
    for tile, crs in mismatched_tiles:
        print(f"{tile}: {crs}")
    print("Please reproject these tiles to NAD83 (e.g., EPSG:4269) using gdalwarp:")
    print("Example: gdalwarp -t_srs EPSG:4269 input.tif output.tif")
else:
    print("\nAll tiles are in NAD83 (EPSG:4269, 26916, or 26917).")

## Step 2: Create VRT for Original DEMs

Create a Virtual Raster Tile (VRT) to reference all 104 DEM tiles as a single dataset, avoiding memory-intensive merging.

In [ ]:
# Create VRT for original DEMs
print("Creating VRT for original DEMs...")
call(['gdalbuildvrt', original_vrt, f"{dem_dir}/*.tif"])
if os.path.exists(original_vrt):
    print(f"VRT created: {original_vrt}")
else:
    print("Error: VRT creation failed.")

## Step 3: Fill Sinks (Pits and Depressions)

Process each DEM tile individually to fill pits and depressions using `richdem`. This ensures a hydrologically conditioned DEM for watershed delineation. Outputs are saved with LZW compression to reduce disk usage.

In [ ]:
# Fill depressions tile-by-tile
for dem_file in tqdm(dem_files, desc="Filling depressions"):
    # Load DEM
    dem = rd.LoadGDAL(dem_file)
    
    # Fill depressions
    rd.FillDepressions(dem, in_place=True)
    
    # Save filled DEM with LZW compression
    output_file = os.path.join(filled_dir, os.path.basename(dem_file))
    rd.SaveGDAL(output_file, dem, creation_options=['COMPRESS=LZW'])

print(f"Filled DEMs saved to: {filled_dir}")

## Step 4: Verify Filled DEMs

Check a sample filled DEM to ensure no depressions remain.

In [ ]:
# Verify one filled DEM
filled_files = glob.glob(f"{filled_dir}/*.tif")
if filled_files:
    sample_file = filled_files[0]
    dem = rd.LoadGDAL(sample_file)
    depressions = rd.IdentifyDepressions(dem)
    if not depressions.any():
        print(f"Verification passed: No depressions in {sample_file}")
    else:
        print(f"Warning: Depressions remain in {sample_file}")
else:
    print("Error: No filled DEMs found.")

## Step 5: Create VRT for Filled DEMs

Create a VRT for the filled DEMs, which will be used for watershed delineation in the next notebook.

In [ ]:
# Create VRT for filled DEMs
print("Creating VRT for filled DEMs...")
call(['gdalbuildvrt', filled_vrt, f"{filled_dir}/*.tif"])
if os.path.exists(filled_vrt):
    print(f"VRT created: {filled_vrt}")
    print("Ready for watershed delineation in the next notebook.")
else:
    print("Error: VRT creation failed.")

## Notes
- **Memory Usage**: Tile-by-tile processing keeps memory usage low, suitable for 8 GB RAM.
- **Disk Space**: Ensure 15 GB free for input (5.57 GB) and output files (~5–7 GB).
- **Next Steps**: Use `ohio_dem_filled.vrt` in the next notebook for flow direction and watershed delineation.
- **Troubleshooting**: If memory errors occur, reduce `richdem`’s memory footprint by processing smaller tiles or using TauDEM (`PitRemove`).

Save this notebook and proceed to the watershed delineation notebook.